<a href="https://colab.research.google.com/github/mfaria-p/Webscrapping_zeolites/blob/main/WebScrapper_LPQ_entregue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#WebScrapper Code

Site: https://europe.iza-structure.org/IZA-SC/ftc_table.php

This Python routine gathers and processes data from HTML pages to extract relevant information about zeolites. It cleans and structures tabular data, retrieves specific text elements from web pages using BeautifulSoup, and formats extracted values for inclusion in a final dataframe. The code includes handling of missing values, string formatting, and HTTP requests to ensure accurate and structured data collection.

In [ ]:
import pandas as pd # usado maioritariamente para ler codigo html (encontra codigo html com a tag 'table') e transforma as tabelas em pandas dataframes
import numpy as np # usei quando recorri a arrays
from bs4 import BeautifulSoup as bs   # para dps organizar e estruturar o código html recebido com o get request, bem como procurar tags e text nesse codigo
import requests # para fazer um HTTP get request para o servidor, para obtermos o codigo html da pagina

In [ ]:
pd.read_html?
# read html tables into a 'list' pf 'DataFrame' objects
# é usado para ler tabelas em sites e transforma as em dataframes
# se houver mais do que uma tabela num site, o codigo vai transcrever todas as tabelas para uma lista
# n preciso de fzr request em HTTP( nao preciso de usar o requests) pk este codigo automaticamente le tabelas em html

In [ ]:
url="https://europe.iza-structure.org/IZA-SC/ftc_table.php"
# Lê a tabela correspondente aos labels dos zeolitos
df3 = pd.read_html(url)[1]

# aqui passo o dataframe obtido para apenas uma lista, por ordem alfabetica
code=[]
for i in range(df3.shape[0]):
  for j in range(df3.shape[1]):
    code.append(df3[j][i])

# retirei os ultimos 4 valores da lista, que eram null
code=code[:-4]

# ao s meter columns, o pandas percebe automaticamente q eu quero coluna
dfcode=pd.DataFrame(data=code, columns=['Zeolite Types'])

# propriedades particulares dos zeolitos- parte do link padrao
url3='https://europe.iza-structure.org/IZA-SC/framework.php?STC='
# informacoes mais particulares acerca dos T-átomos de cada zeólito
url4='https://europe.iza-structure.org/IZA-SC/framework_coord.php?STC='
# material de referencia para cada zeolito, como a sua formula quimica, dimensoes dos canais, etc
url5='https://europe.iza-structure.org/IZA-SC/material_tm.php?STC='

#Têm que s tirar o menos aos zeolitos q têm um menos(-) antes da sigla, pk para o link do site estes não o têm
propriedades=[]
for i in range(dfcode.shape[0]):
  if (dfcode['Zeolite Types'][i])[0]=='-':
    zeolite=(dfcode['Zeolite Types'][i])[1:]
  else:
    zeolite=dfcode['Zeolite Types'][i]

  # lê se o link para cada zeolito (informações gerais)
  link=f"{url3}{zeolite}"
  # lê se o link para cada zeolito (agora para a lista dos T-atomos, as coordenadas e mais informacoes acerca destes)
  link2=f"{url4}{zeolite}"
  # lê se o link para cada zeolito (agora para materiais de referencia, como a sua formula quimica)
  link3=f"{url5}{zeolite}"

  # A segunda tabela em todas as paginas de todos os zeolitos contém info acerca do framework density e dos ring sizes
  tabela1=pd.read_html(link)[2]

  # A quarta tabela em todas as páginas de todos os zeólitos contém info acerca do max diameter of a sphere that can be included
  tabela2=pd.read_html(link)[4]

  # A quinta tabela em todas as páginas de todos os zeolitos contém info acerca do max diameter of a sphere that can diffuse along
  tabela3=pd.read_html(link)[5]

  # A sexta tabela em todas as páginas de todos os zeolitos contém info acerca do accessible volume
  tabela4=pd.read_html(link)[6]


  # A tabela que contém o natural tiling de cada zeólito varia de localização na pagina de cada um
  # Assim tive primeiro que encontrar a tabela certa de onde retirar os dados
  # A tabela tem sempre o mesmo formato, ou seja o 'natural Tiling' encontra-se sempre no mesmo sitio
  # Coloquei o data frame em array pk foi-me mais facil visualizar e procurar o termo 'Natural Tiling'
  for j in range(8,13):
    natil=pd.read_html(link)[j]
    array = (natil).to_numpy()

    if (len(array)>1 and len(array[1])>1):
      if array[1][1]=='Natural Tiling':
        break
      else:
        continue
    else:
      continue

# Na tabela do 'Natural Tiling', tive ainda que retirar todos os valores nulos.
# Neste caso, não me foi possivel usar dropna() pk isso retira linhas com valores nulos e havia linhas com valores nulos, mas também
#  valores do natural tiling
# como o formato da tabela era smp o mesmo:
  tabela5=[]

# Na segunda linha, tinhamos o natural tiling e em seguida valores
  for j in range(2,(natil).shape[1]-1):
    tabela5.append((natil)[j][1])

# se houvessem muitos valores de natural tiling então às vezes o dataframe passava para uma outra linha e nesse caso
# só queria os valores não nulos
  if (natil).shape[0]>2:
    for j in range(2,(natil).shape[1]-1):
      if ((natil).isna())[j][2]==True:
        exit()
      else:
        tabela5.append((natil)[j][2])

  if (natil).shape[0]>3:
    for j in range(2,(natil).shape[1]-1):
      if ((natil).isna())[j][3]==True:
        exit()
      else:
        tabela5.append((natil)[j][3])

  # Eu queria que no meu dataframe final, o resultado aparecesse em formato de valor1,valor2 , e não sob a forma de uma lista,
  #  portanto passei para string a lista e retirei os caracteres que não queria que aparecessem
  tabela5=str(tabela5)
  tabela5=[tabela5[i] for i in range(len(tabela5)) if (tabela5[i]!="'" and tabela5[i]!='[' and tabela5[i]!=']')]
  tabela5=''.join(tabela5)


  # Para os subtítulos das imagens do Composite Building Units, procurar a tabela e colocá la em dataframe para retirar os valores
  #  desformatava depois esses subtitulos então tive que recorrer a outra forma, nomeadamente a de retirar diretamente do código html

  # Primeiro tive que que enviar um HTTP request ao server, para ele me responder com o content of the webpage(em html)
  r=requests.get(link)
  # Só que esse codigo dá-nos o raw content of the webpage, isto é, vem sob a forma de string tudo

  # Através da BeautifulSoup, organizo o raw content para se conseguir analisar o codigo html dps (estrutura o codigo html que estava em string)
  soup=bs(r.content,'html5lib')

  # Depois, através da leitura do codigo html, consegui descobrir os atributos especificos ta tabela que eu procuro,
  #  atributos que mais nenhuma tabela da pagina tinha
  # entao obtenho o codigo para a tabela dos Composite Building Units
  tabelacbu = soup.find('table', attrs={'width':"650", 'border':"0"})

  # Dentro dessa tabela, obtenho a parte do codigo que contém a parte string que eu quero obter
  #  (encontrei-a pelos seus atributos especificos)
  cbu=tabelacbu.findAll('div',attrs = {'align':"center"})

  # Agora precisei de retirar a parte string desse codigo, então recorro ao comando .text
  tabela6=[]
  for row in cbu:
        if row.text=='':
          continue
        row_text = row.text
        tabela6.append(row_text)

  # Há zeolitos que não têm nada nas composite building units, portanto coloquei que nesses casos iria aparecer '---' no dataframe final
  if tabela6==[]:
    tabela6=['---']

  # Eu queria que no meu dataframe final, o resultado aparecesse em formato de valor1,valor2 , e não sob a forma de uma lista,
  #  portanto passei para string a lista e retirei os caracteres que não queria que aparecessem
  tabela6=str(tabela6)
  tabela6=[tabela6[i] for i in range(len(tabela6)) if (tabela6[i]!="'" and tabela6[i]!='[' and tabela6[i]!=']')]
  tabela6=''.join(tabela6)


  # A seguir, quis retirar a propriedade do space group, relativamente à distribuicao dos T atomos
  # Primeiro tive que que enviar um HTTP request ao server, para ele me responder com o content of the webpage(em html)
  r=requests.get(link2)
  # Só que esse codigo dá-nos o raw content of the webpage, isto é, vem sob a forma de string tudo

  # Através da BeautifulSoup, organizo o raw content para se conseguir analisar o codigo html dps (estrutura o codigo html que estava em string)
  soup=bs(r.content,'html5lib')

  # Depois, através da leitura do codigo html, consegui descobrir os atributos especificos da tabela que eu procuro,
  #  atributos que mais nenhuma tag da pagina tinha
  # entao obtenho o codigo para a parte deste que contém as informacoes acerca do space group
  tabela = soup.find('div', attrs={'class':"SubheadNorm14"})

  # aqui obtenho a parte do codigo que tem o space group especifico de cada zeolito
  sgp=tabela.find('i')
  # passo para string, para dps poder tirar as partes que não me intererssam(não retiro automaticamente logo
  #  o texto pk ha comandos dentro do text tipo sub que preciso de valorizar para o space group ficar bem escrito)
  sgp=str(sgp)

  # Se tiver um sub no codigo então significa q é um numero em baixo de outro e meto um '_' para ser de mais facil leitura
  if 'sub' in sgp:
    spacegroup=sgp.replace('<sub>','_')
    spacegroup1=spacegroup.replace('<i>',' ')
    spacegroup2=spacegroup1.replace('</i>',' ')
    spacegroup3=spacegroup2.replace('</sub>','')
  # se não tiver sub, então é so retirar o resto do codigo para obter a string normal
  else:
    spacegroup=sgp.replace('<i>',' ')
    spacegroup3=spacegroup.replace('</i>',' ')

  tabela7= spacegroup3


  # Agora queria retirar os comprimentos a, b e c e os angulos do zeolito para formar mais tres colunas
  tabela8=pd.read_html(link2)[0]

  # todas as primeiras tabelas da pagina de cada zeolito tinha a informacao da formula quimica
  formula=pd.read_html(link3)[0]

  # Havia casos em que a formula quimica era obtida com informacoes à frente. O texto da parte desnessária e associada a imagens(este começa com a palavra 'SMILES') é retirada
  # nestes casos. A informacao acerca de compostos dentro da formula quimica (que aparece em conjunto com o texto 'SMILES') de cada zeolito é retirada para ser colocada numa
  # outra coluna denominada por 'informacoes adicionais'. Caso não haja info. adicional a ser acrescentada, entao é colocado um '---'
  if 'SMILES' in formula[2][1]:
    tabela9=formula[2][1].split('SMILES')[0]
    tabela9=tabela9.split('-'+ str(dfcode['Zeolite Types'][i]))
  else:
    tab=formula[2][1].replace('-'+ str(dfcode['Zeolite Types'][i]),'')
    tabela9=[tab,'---']

  if '[1]' in tabela9:
    tab1=tabela9.replace('[1]','')
    tabela9=tab1


  # todas as segundas tabelas da pagina de cada zeolito tinha as informacoes dos canais
  channels=pd.read_html(link3)[1]

  # Da info obtida, retirei o texto não pretendido (este começava sempre com a palavra 'Dimensionality')
  tabela10=channels[2][0].split('Dimensionality')[0]

  if tabela10=='': tabela10='---'

  # Ainda retirei a Unit Cell Shape de cada zeólito
  # Primeiro tive que que enviar um HTTP request ao server, para ele me responder com o content of the webpage(em html)
  r=requests.get(link)
  # Só que esse codigo dá-nos o raw content of the webpage, isto é, vem sob a forma de string tudo

  # Através da BeautifulSoup, organizo o raw content para se conseguir analisar o codigo html dps (estrutura o codigo html que estava em string)
  soup=bs(r.content,'html5lib')

  # encontro o elemento que contém o texto 'Cell Parameters'
  elemento = soup.find('td',string='Cell Parameters:')
  # Encontro o seu 'irmao', isto é, a linha de codigo correspondente que vem a seguir
  tabela11 = elemento.find_next_sibling('td')
  tabela11=str(tabela11.text)


 #Criei uma lista (propriedades) de listas
  propriedades.append([tabela11,tabela7,(tabela8[0])[1],tabela8[1][1],tabela8[2][1],(tabela8[3])[1],tabela8[4][1],
                       tabela8[5][1],(tabela1[2])[0],(tabela1[2])[2],tabela10,
                       (tabela2[2])[0],[(tabela3[2])[0],(tabela3[3])[0],(tabela3[4])[0]],(tabela4[2])[0],tabela5,tabela6,tabela9[0],tabela9[1]])
# propriedades: (tabela[2])[0] é o framework density, (tabela[2])[2] é os ring sizes, (tabela2[2])[0] é o max. diameter of a sphere that can be included,
#  [(tabela3[2])[0],(tabela3[3])[0],(tabela3[4])[0]]são os valores do max. diameter of a sphere that can diffuse along, (tabela4[2])[0] é o accessible volume
# tabela5 é o Natural Tiling, tabela6 são as composite building units, tabela7 é o space group para os T atomos
# (tabela8[0])[1],tabela8[1][1],tabela8[2][1] sao os comprimentos a,b e c do zeolito, respetivamente
# (tabela8[3])[1],tabela8[4][1],tabela8[5][1] são os angulos alfa, beta e gama, respetivamente
# tabela9[0] é a formula quimica do composto e tabela9[1] são as info adicionais acerca do composto (caso este as tenha)
# tabela10 são as informacões acerca dos canais dos zeolitos
# tabela11.text é a parte string do código que retirei (tabela11), é a Unit Cell Shape

for i in range(len(propriedades)):
# tirei as unidades do framework density e do max diameter of a sphere that can be included para tar la so o valor
  propriedades[i][8]=propriedades[i][8].split(' ')[0]
  propriedades[i][11]=str(propriedades[i][11]).split('\xa0')[0]

# Dos tres valores obtidos na tabela do max diameter of a sphere that can diffuse along, tive que os juntar, sem as unidades,
#  e separados por ponto e virgula
  propriedades[i][12]=str(propriedades[i][12][0]).split('\xa0')[0]+'; '+str(propriedades[i][12][1]).split('\xa0')[0]+'; '+str(propriedades[i][12][2]).split('\xa0')[0]

# Para o accessible volume tambem tive que retirar as unidades
  propriedades[i][13]=propriedades[i][13].split(' ')[0]

  # Dos tres valores obtidos para os comprimentos a,b e c, tive que lhes tirar as unidades e a parte de 'a = '...
  propriedade1=str(propriedades[i][2]).replace('a = ','')
  propriedades[i][2]=propriedade1
  propriedades[i][2]=str(propriedades[i][2]).split(' ')[0]

  propriedade2=propriedades[i][3].replace('b = ','')
  propriedades[i][3]=propriedade2
  propriedades[i][3]=str(propriedades[i][3]).split(' ')[0]

  propriedade3=propriedades[i][4].replace('c = ','')
  propriedades[i][4]=propriedade3
  propriedades[i][4]=str(propriedades[i][2]).split(' ')[0]

  # Dos tres valores obtidos para as amplitudes dos angulos alfa,beta e gama, tive que tirar as unidades e a parte de 'α = '...
  propriedade1=str(propriedades[i][5]).replace('α = ','')
  propriedades[i][5]=propriedade1
  propriedades[i][5]=str(propriedades[i][5]).split(' ')[0]

  propriedade2=propriedades[i][6].replace('β = ','')
  propriedades[i][6]=propriedade2
  propriedades[i][6]=str(propriedades[i][6]).split(' ')[0]

  propriedade3=propriedades[i][7].replace('γ = ','')
  propriedades[i][7]=propriedade3
  propriedades[i][7]=str(propriedades[i][7]).split(' ')[0]


# Juntei tudo num DataFrame, com as propriedades como diferentes colunas
dpropriedades=pd.DataFrame(data=(propriedades),columns=['Unit Cell Shape',
                                                        'Space Group',
                                                        'a (Å) ',
                                                        'b (Å) ',
                                                        'c (Å) ',
                                                        'α (°) ',
                                                        'β (°) ',
                                                        'γ (°) ',
                                                        'Framework density (T/1000 Å^3)',
                                                        'Ring sizes (# T-atoms)',
                                                        'Channels',
                                                        'Max. diameter of a sphere, that can be included (Å)',
                                                        'Max. diameter of a sphere, that can diffuse along (Å)',
                                                        'Accessible Volume (%)',
                                                        'Natural Tiling ',
                                                        'Composite Building Units',
                                                        'Chemical Formula',
                                                        'Additional Information'
                                                        ])


# concatenei o primeiro DataFrame que tinha criado (os labels dos zeolitos em ordem alfabetica), com as suas propriedades
# axis=1 significa que junta os Dataframes como colunas
result=pd.concat([dfcode,dpropriedades],axis=1)
display(result)
result.to_excel('/content/zeolitos_o_tal.xlsx')

,Zeolite Types,Unit Cell Shape,Space Group,a (Å),b (Å),c (Å),α (°),β (°),γ (°),Framework density (T/1000 Å^3),Ring sizes (# T-atoms),Channels,"Max. diameter of a sphere, that can be included (Å)","Max. diameter of a sphere, that can diffuse along (Å)",Accessible Volume (%),Natural Tiling,Composite Building Units,Chemical Formula,Additional Information
0,ABW,orthorhombic,I m m a,9.8730,5.2540,9.8730,90.000°,90.000°,90.000°,17.6,8 6 4,[001] 8 3.4 x 3.8*,4.24,a: 2.61; b: 3.51; c: 2.61,5.19,t-kdq,abw (t-kdq ),|Li4 (H2O)4 | [Al4Si4O16],---
1,ACO,cubic,I m -3 m,9.9050,9.9050,9.9050,90.000°,90.000°,90.000°,16.5,8 4,<100> 8 2.8 x 3.5** <-> [001] 8 3.5 x 3.5*,4.58,a: 3.56; b: 3.56; c: 3.56,8.68,"t-cub, t-ste",d4r (t-cub ),|(EDA)16 (H2O)8| [Al0.88Co7.12P8O32],"EDA = C2H8N2 = ethylenediamine = ethane-1,2-d..."
2,AEI,orthorhombic,C m c m,13.6770,12.6070,13.6770,90.000°,90.000°,90.000°,15.1,8 6 4,{[100] 8 3.8 x 3.8 <-> [110] 8 3.8 x 3.8 <-> [...,7.33,a: 3.84; b: 3.84; c: 3.64,17.31,"t-hpr, t-per",d6r (t-hpr ),[Al24P24O96],---
3,AEL,orthorhombic,I m m a,8.3120,18.7290,8.3120,90.000°,90.000°,90.000°,19.2,10 6 4,[001] 10 4.0 x 6.5*,5.64,a: 4.63; b: 2.01; c: 2.01,6.77,"t-afi, t-kah, t-lov, t-odp","afi (t-afi ) , bog",[Al20P20O80],---
4,AEN,orthorhombic,C m c e,18.5310,13.3550,18.5310,90.000°,90.000°,90.000°,20.1,8 6 4,[100] 8 3.1 x 4.3* <-> [010] 8 2.7 x 5.0*,4.43,a: 1.21; b: 3.38; c: 3.66,4.02,"t-cor, t-kah",---,|(EDA)16 (H2O)64| [Al24P24O96],"EDA = C2H8N2 = ethylenediamine = ethane-1,2-d..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,WEI,orthorhombic,C c c m,11.8030,10.2780,11.8030,90.000°,90.000°,90.000°,16.5,10 8 6 4 3,[001] 10 3.1 x 5.4* <-> [100] 8 3.3 x 5.0*,4.19,a: 3.43; b: 3.43; c: 3.46,7.82,t-sfg,"lov, vsv",|Ca4 (H2O)16 | [Be12P8H8O40],---
247,-WEN,hexagonal,P -6 2 m,13.5870,13.5870,13.5870,90.000°,90.000°,120.000°,16.6,10 8 6 4,<100> 10 2.5 x 4.8** <-> [001] 8 2.3 x 2.7*,5.53,a: 4.84; b: 4.84; c: 3.16,11.58,"t-can, t-dmp, t-hpr, t-srs, t-wen","d6r (t-hpr ) , can (t-can )","|Ba4(Ca,Na2)3 (SO4)3 H2O | [Al8Si12O39(OH)2]",---
248,YFI,orthorhombic,C m m m,18.1807,31.8406,18.1807,90.000°,90.000°,90.000°,16.4,12 8 6 5 4,[100] 12 6.0 x 7.8 <-> [010] 12 6.0 x 7.2 <-> ...,7.97,a: 6.18; b: 6.18; c: 3.6,15.37,"t-kah, t-tes, t-mel, t-umx, t-bea-1, t-mtw, t-...","mor (t-tes ) , mtt, mel, mtw (t-mtw ) , d8r (t...",|K6.3 (H2O)0.72 | [Al12Si108O240],---
249,YUG,monoclinic,C 1 2/m 1,10.2480,13.7880,10.2480,90.000°,111.545°,90.000°,18.0,8 5 4,[100] 8 2.8 x 3.6* <-> [001] 8 3.1 x 5.0*,4.49,a: 3.08; b: 1.44; c: 3.12,5.70,"t-kdk, t-ygr",---,|Ca2 (H2O)8 | [Al4Si12O32],---
